In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By 

# Beautiful Soup 
from bs4 import BeautifulSoup

# Automates downloading and setting up of Chrome Driver
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
def scrape_page(driver, current_url: str) -> list: 
    """    
    This functions scrapes a single webpage in a paginated website. 

    Args: 
    driver: 
    current_url:

    Returns: List of products scraped from a single page
    """ 

    pass 

def main(BASE_URL: str, num_pages: str): 
    """   
    This is the main function that will be used to scrape the entire website 

    """ 
    pass

In [12]:
BASE_URL = "https://www.kilimall.co.ke/category/shoes?id=968&form=category&page=55"

# Downloading and setting up Chrome Driver 
service = Service(ChromeDriverManager().install()) 

# initializing the webdriver 
driver = webdriver.Chrome(service=service) 

# Navigating to the website 
driver.get(BASE_URL) 

# Let the scrapper wait for 5 seconds to ensure page has fully loaded 
wait = WebDriverWait(driver, 5)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body"))) 

# Access the HTML content 
html_markup = driver.page_source 
soup = BeautifulSoup(html_markup, "html.parser") 

# Get all the tags containing the products listing 
products_wrapper = soup.find_all("div", {"class": "listing-item"})

# Loop through the products_wrapper 
for product in products_wrapper: 
    product_name = product.find("div", {"class": "product-price"}).text 

driver.quit()